# 데이터소스
- 스파크 핵심 데이터소스 
  - CSV
  - JSON
  - 파케이
  - ORC
  - JDBC/ODBC연결
  - 일반 텍스트 파일

- 핵심 서드파티 데이터소스
  - 카산드라
  - HBase
  - 몽고디비
  - AWS Redshift
  - XML

In [0]:
path='/FileStore/tables/all/*.csv'

## 데이터소스 API의 구조

### 읽기 API 구조
- 스파크에서 데이터를 읽을 땐 기본적으로 <strong>DataFrameReader</strong>를 사용함
- DataFrameReader는 <strong>SparkSession의 read 속성</strong>으로 접근
- DataFrameReader를 얻고 나서는 다음과 같은 값을 지정해야함
  - format: 포맷 지정(default: 파케이)
  - option: 데이터 읽는 방법 지정 ex)키-값 쌍이면 option('key', 'value')
  - schema: 데이터 소스에서 스키마를 제공하거나 스키마 추론 기능을 사용하려고 할 때 지정

In [0]:
spark.read

Out[1]: <pyspark.sql.readwriter.DataFrameReader at 0x7f6b4e3e9f40>

In [0]:
df = spark.read.format('csv')\
.option('mode', 'FAILFAST')\
.option('inferSchema', 'true')\
.option('path', path)\
.load()

---
- 위 예시에서는 읽기 모드를 'failfast'로 지정함
- <strong>읽기 모드: 스파크가 형식에 맞지 않는 데이터를 만났을 때의 동작 방식을 지정하는 옵션</strong>
  - permissive(default): 오류 레코드의 모든 필드를 null로 설정하고 모든 오류 레코드를 _corrupt_record라는 문자열 컬럼에 기록
  - dropMalformed: 형식에 맞지 않는 레코드가 포함된 로우 제거
  - failFast: 형식에 맞지 않는 레코드를 만나면 즉시 종료

### 쓰기 API 구조
- 데이터 읽기와 매우 유사한데 DataFrameReader 대신 <strong>DataFrameWriter</strong>를 사용
- 데이터소스에 항상 데이터를 기록해야하므로 <strong>DataFrame의 write속성</strong>을 사용
  - DataFrame별로 DataFrameWriter에 접근
- DataFrameWriter를 얻고나서는 다음과 같은 값을 지정해야함
  - format
  - option
  - 파일 기반의 데이터소스만 해당
    - partitionBy
    - bucketBy
    - sortBy

In [0]:
df.write.format('csv')\
.option('mode', 'OVERWRITE')\
.option('dateFormat', 'yyyy-MM-dd')\
.option('path', '/FileStore/tables/temp/temp.csv')\
.save()

- 위 예시에서는 저장 모드를 'overwrite'로 지정함
- 저장 모드: 스파크가 지정된 위치에서 동일한 파일이 발견됐을 때의 동작 방식 지정
  - append: 해당 경로에 이미 존재하는 파일 목록에 결과 파일 추가
  - overwrite: 이미 존재하는 모든 데이터를 완전히 덮어씀
  - errorIfExists(default): 오류를 발생시키면서 쓰기 작업이 실패
  - ignore: 아무런 처리 X

## CSV 파일
- CSV(comma-separated values)는 <strong>콤마(,)로 구분된 값</strong>을 의미
- 각 줄이 단일 레코드
- 각 필드를 콤마로 구분하는 일반적인 텍스트 파일 포맷
- 해당 포맷이 다루기 까다로운 이유
  - 운영 환경에서는 어떤 내용, 어떤 구조로 되어 있는지 등 다양한 전제를 만들어낼 수 없기 때문
  - 그래서 <strong>CSV reader는 많은 수의 옵션</strong>을 제공
  - [링크](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html?highlight=csv#pyspark.sql.DataFrameReader.csv)

### CSV 파일 읽기

In [0]:
df =spark.read.format('csv')\
.option('header', 'true')\
.option('mode','FAILFAST')\
.load(path)

In [0]:
df.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
 536365| 85123A|WHITE HANGING HEA...| 6|12/1/2010 8:26| 2.55| 17850|United Kingdom|
 536365| 71053| WHITE METAL LANTERN| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|
 536365| 84406B|CREAM CUPID HEART...| 8|12/1/2010 8:26| 2.75| 17850|United Kingdom|
 536365| 84029G|KNITTED UNION FLA...| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|
 536365| 84029E|RED WOOLLY HOTTIE...| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows

### CSV 파일 쓰기

In [0]:
df.write.format('csv').mode('overwrite').option('sep','\t').save('/FileStore/tables/temp/tsv_file.tsv')

-----
- 위처럼 csv파일을 읽어 들여 tsv파일로 내보는 처리도 간단

## JSON 파일
- <strong>자바스크립트 객체 표기법(JavaScript Object Notation)</strong>
- 스파크에서는 JSON파일을 사용할 때 <strong>줄로 구분된 JSON</strong>을 기본적으로 사용
  - <strong>큰 JSON 객체나 배열을 하나씩 가지고 있는 파일</strong>을 다루는 것과 대조적인 부분
  - <strong>multiLine옵션</strong>으로 바꿀 순 있음
  - 근데 줄로 구분된 방식이 더 안정적이라서 디폴트임
    - 구조화 되어 있음
    - 최소한의 기본 데이터 타입이 존재함
    
- JSON은 객체이므로 CSV보다 옵션 수가 적음
  - [링크](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html?highlight=json#pyspark.sql.DataFrameReader.json)

### JSON 파일 읽기

In [0]:
j_path='/FileStore/tables/2010_summary.json'
spark.read.format('json').option('mode', 'FAILFAST').option('inferSchema', 'true').load(j_path).show(5)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 1|
 United States| Ireland| 264|
 United States| India| 69|
 Egypt| United States| 24|
Equatorial Guinea| United States| 1|
+-----------------+-------------------+-----+
only showing top 5 rows

### JSON 파일 쓰기

In [0]:
df.write.format('json').mode('overwrite').save('/FileStore/tables/temp/j_file.json')

-----
- 데이터소스에 관계없이 json파일에 저장할 수 있음
- 위처럼 csv DataFrame을 json파일의 소스로 재사용 가능

## 파케이 파일
- 다양한 스토리지 최적화 기술을 제공하는 오픈소스로 만들어진 <strong>컬럼 기반의 데이터 저장 방식</strong>
- 저장 공간 절약 가능
- <strong>전체 파일을 읽는 대신 개별 컬럼</strong>을 읽음
- 컬럼 기반 압축 기능 제공
- 아파치 스파크와 호환이 잘 됨
  - 그래서 스파크의 기본 파일 포맷
- json이나 csv보다 읽기 연산이 더 효율적
- 장기 저장용 데이터는 파케이 포맷으로 저장하는 것이 좋음
- 복합 데이터 타입 지원
  - csv에서는 배열을 사용할 수 없음
- 옵션이 거의 없음(2개)
  - 스파크 개념에 아주 잘 부합하고 알맞게 정의된 명세를 가지고 있기 때문
  - [링크](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html?highlight=csv#pyspark.sql.DataFrameReader.parquet)

### 파케이 파일 읽기

In [0]:
p_path='/FileStore/tables/2010-summary.parquet'
spark.read.format('parquet').load(p_path).show(5)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 1|
 United States| Ireland| 264|
 United States| India| 69|
 Egypt| United States| 24|
Equatorial Guinea| United States| 1|
+-----------------+-------------------+-----+
only showing top 5 rows

### 파케이 파일 쓰기

In [0]:
df.write.format('parquet').mode('overwrite').save('/FileStore/tables/temp/p_file.parquet')

---
- 다른 포맷과 동일

## ORC 파일
- <strong>ORC는 하둡 워크로드를 위해 설계된 자기 기술적(self-describing)이며 데이터 타입을 인식할 수 있는 컬럼 기반의 파일 포맷</strong>
- 대규모 스트리밍 읽기에 최적화
- 필요한 로우를 신속하게 찾아낼 수 있음
- 스파크는 ORC파일 포맷을 별도의 옵션 지정 없이 효율적으로 사용 가능
- 파케이와의 차이?
  - 파케이는 스파크에 최적화
  - ORC는 하이브에 최적화

### ORC 파일 읽기

In [0]:
o_path='/FileStore/tables/2010-summary.orc/part_r_00000_2c4f7d96_e703_4de3_af1b_1441d172c80f_snappy.orc'
spark.read.format('orc').load(o_path).show(5)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 1|
 United States| Ireland| 264|
 United States| India| 69|
 Egypt| United States| 24|
Equatorial Guinea| United States| 1|
+-----------------+-------------------+-----+
only showing top 5 rows

### ORC 파일 쓰기

In [0]:
df.write.format('orc').mode('overwrite').save('/FileStore/tables/temp/o_file.orc')

## SQL 데이터베이스
- 매우 강력한 <strong>커넥터</strong> 중 하나
- SQL을 지원하는 다양한 시스템에 SQL 데이터소스 연결
  - MySQL
  - PostgreSQL
  - Oracle
  - SQLite
- 데이터 베이스는 원시 파일 형태가 아니므로 고려해야할 옵션이 많음
  - 데이터 베이스 인증 정보나 접속 관련 옵션
  - 스파크 클러스터에서 데이터 베이스 시스템에 접속 가능한지 네트워크 상태 확인

### SQL 데이터베이스 읽기
- 다른 데이터소스처럼 포맷과 옵션을 지정한 후 데이터를 읽어들임

In [0]:
driver='org.sqlite.JDBC'
s_path='dbfs:/FileStore/tables/my_sqlite.db'
url= 'jdbc:sqlite:'+s_path
tablename='flight_info'

In [0]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/2010-summary.orc/,2010-summary.orc/,0
dbfs:/FileStore/tables/2010-summary.parquet/,2010-summary.parquet/,0
dbfs:/FileStore/tables/2010_summary.csv,2010_summary.csv,7121
dbfs:/FileStore/tables/2010_summary.json,2010_summary.json,21353
dbfs:/FileStore/tables/all/,all/,0
dbfs:/FileStore/tables/my_sqlite.db,my_sqlite.db,11264
dbfs:/FileStore/tables/my_sqlite_copy.db,my_sqlite_copy.db,11264
dbfs:/FileStore/tables/temp/,temp/,0


In [0]:
#파일 경로 문제 참조
#https://stackoverflow.com/questions/68202341/analysisexception-path-does-not-exist-dbfs-databricks-python-lib-python3-7-si
#https://docs.databricks.com/dev-tools/databricks-utils.html
copy_path='/FileStore/tables/my_sqlite_copy.db'
new_path= '/tmp/my_sqlite_copy.db'
dbutils.fs.cp(copy_path, new_path)

In [0]:
%fs ls /tmp

path,name,size
dbfs:/tmp/hive/,hive/,0
dbfs:/tmp/my_sqlite_copy.db,my_sqlite_copy.db,11264


In [0]:
url= 'jdbc:sqlite:'+new_path

In [0]:
url

Out[8]: 'jdbc:sqlite:/tmp/my_sqlite_copy.db'

In [0]:
dbDF = spark.read.format('jdbc').option('url', url).option('dbtable',tablename).option('driver', driver).load()

---
- 그냥 이렇게하고 일반 데이터프레임처럼 쓰면 됨
  - 근데 자꾸 에러가...

#### 쿼리푸시다운
- 스파크는 DataFrame을 만들기 전에 DB자체에서 데이터를 필터링하도록 만들 수 있음

In [0]:
dbDF.filter("DEST_COUNTRY_NAME in ('Anguilla', 'Sweden')")

----
- 위 처럼 필터를 명시하면 스파크는 해당 필터에 대한 처리를 DB에 위임한다.
 - 위임하는걸 push down이라고 함

In [0]:
pushdownQuery="(select distinct(dest_country_name) from flight_info) as flight_info"
dbDF = spark.read.format('jdbc').option('url', url).option('dbtable',pushdownQuery).option('driver', driver).load()

----
- 또 위처럼 전체 쿼리를 DB에 직접 전달해서 DataFrame으로 결과를 받아야하는 경우엔 테이블명 대신 SQL 쿼리를 명시하면 됨

#### 데이터베이스 병렬로 읽기
- 스파크는 파일 크기, 파일 유형, 압축 방식에 따른 <strong>'분할 가능성'</strong>에 따라
  - 여러 파일을 읽어 하나의 파티션으로 만들거나 
  - 여러 파티션을 하나의 파일로 만드는 알고리즘을 가짐

- 파일이 가진 이런 유연성은 SQL 데이터베이스에도 존재하지만 몇 가지 수동 설정이 필요
  - 옵션 목록 중 <strong>numPartitions</strong> 옵션으로 읽기 및 쓰기용 <strong>동시 작업 수를 제한할 수 있는 최대 파티션 수</strong> 설정 가능

In [0]:
dbDF = spark.read.format('jdbc').option('url', url).option('dbtable',tablename).option('driver', driver)\
.option('numPartitions',10).load()

#### 슬라이딩 윈도우 기반의 파티셔닝
- 조건절을 기반으로 분할할 수 있는 방법

In [0]:
props ={'driver':'org.sqlite.JDBC'}
#분할 기준 컬럼
colName= 'count' 
#처음과 마지막 파티션 사이의 최솟값과 최댓값 (이 범위 밖의 모든 값은 첫 번째 또는 마지막 파티션에 속함)
lowerBound=0
upperBound=348113
#파티션 수
numPartitions=10

In [0]:
spark.read.jdbc(url, tablename, column=colName, properties=props, lowerBound=lowerBound, upperBound=upperBound,numPartitions=numPartitions )

### SQL 데이터베이스 쓰기
- URI를 지정하고 지정한 쓰기 모드에 따라 데이터를 쓰면 됨

In [0]:
newPath='jdbc:sqlite://tmp/new_qlite.db'
df.write.jdbc(newPath, tablename, properties=props )

## 텍스트 파일
- 파일의 각 줄은 DataFrame의 레코드

### 텍스트 파일 읽기

In [0]:
spark.read.format('text').load('/FileStore/tables/2010_summary.csv').selectExpr("split(value,',') as rows").show(5)

+--------------------+
 rows|
+--------------------+
[DEST_COUNTRY_NAM...|
[United States, R...|
[United States, I...|
[United States, I...|
[Egypt, United St...|
+--------------------+
only showing top 5 rows

### 텍스트 파일 쓰기

In [0]:
df.select('StockCode').write.text('/FileStore/tables/tmp.txt')

In [0]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/2010-summary.orc/,2010-summary.orc/,0
dbfs:/FileStore/tables/2010-summary.parquet/,2010-summary.parquet/,0
dbfs:/FileStore/tables/2010_summary.csv,2010_summary.csv,7121
dbfs:/FileStore/tables/2010_summary.json,2010_summary.json,21353
dbfs:/FileStore/tables/all/,all/,0
dbfs:/FileStore/tables/my_sqlite.db,my_sqlite.db,11264
dbfs:/FileStore/tables/my_sqlite_copy.db,my_sqlite_copy.db,11264
dbfs:/FileStore/tables/temp/,temp/,0
dbfs:/FileStore/tables/tmp.txt/,tmp.txt/,0


## 고급 I/O개념
- 쓰기 작업 전에 <strong>파티션 수를 조절</strong>함으로써 병렬로 처리할 파일 수를 제어할 수 있다.
- 또한 <strong>버켓팅과 파티셔닝을 조절</strong>함으로써 데이터의 저장 구조를 제어할 수 있다.

### 분할 가능한 파일 타입과 압축 방식
- <strong>기본적으로 분할을 지원하는 특정 포맷</strong>은 전체 파일이 아닌 쿼리에 <strong>필요한 부분만</strong> 읽을 수 있으므로 성능 향상에 도움이 됨
  - <strong>HDFS</strong> 같은 시스템을 사용한다면 <strong>분할된 파일을 여러 블록으로 나누어 분산 저장</strong>하므로 더 최적화 가능
- <strong>압축 방식</strong>도 관리해야하는데, 모든 압축 방식이 <strong>분할 압축</strong>을 지원하진 않음
- 추천 파일 포맷: 파케이
- 추천 압축 방식: GZIP

### 병렬로 데이터 읽기
- 여러 익스큐터가 같은 파일을 동시에 읽을 순 없지만 <strong>여러 파일을 동시에</strong> 읽을 순 있음
- <strong>다수의 파일이 존재하는 폴더</strong>를 읽을땐 폴더의 <strong>개별 파일은 DataFrame의 파티션</strong>이된다.
- 따라서 사용 가능한 익스큐터를 이용해 병렬로 파일을 읽음
  - 익스큐터 수를 넘어가는 파일은 처리 중인 파일이 완료될 때까지 대기

### 병렬로 데이터 쓰기
- 파일이나 데이터 수는 데이터를 <strong>쓰는 시점에 DataFrame이 가진 파티션 수</strong>에 따라 달라질 수 있음
- 기본적으로 데이터 파티션당 하나의 파일이 작성
- 옵션에 지정된 파일명은 실제로는 다수의 파일을 가진 디렉터리

In [0]:
#폴더 안에 5개의 파일을 생성함
df.repartition(5).write.format('csv').save('/tmp/multiple.csv')

In [0]:
%fs ls /tmp/multiple.csv

path,name,size
dbfs:/tmp/multiple.csv/_SUCCESS,_SUCCESS,0
dbfs:/tmp/multiple.csv/_committed_7308919149863971029,_committed_7308919149863971029,463
dbfs:/tmp/multiple.csv/_started_7308919149863971029,_started_7308919149863971029,0
dbfs:/tmp/multiple.csv/part-00000-tid-7308919149863971029-29eda532-ce72-4f1b-83a7-f0340c44f34f-9-1-c000.csv,part-00000-tid-7308919149863971029-29eda532-ce72-4f1b-83a7-f0340c44f34f-9-1-c000.csv,9040894
dbfs:/tmp/multiple.csv/part-00001-tid-7308919149863971029-29eda532-ce72-4f1b-83a7-f0340c44f34f-10-1-c000.csv,part-00001-tid-7308919149863971029-29eda532-ce72-4f1b-83a7-f0340c44f34f-10-1-c000.csv,9046349
dbfs:/tmp/multiple.csv/part-00002-tid-7308919149863971029-29eda532-ce72-4f1b-83a7-f0340c44f34f-11-1-c000.csv,part-00002-tid-7308919149863971029-29eda532-ce72-4f1b-83a7-f0340c44f34f-11-1-c000.csv,9038812
dbfs:/tmp/multiple.csv/part-00003-tid-7308919149863971029-29eda532-ce72-4f1b-83a7-f0340c44f34f-13-1-c000.csv,part-00003-tid-7308919149863971029-29eda532-ce72-4f1b-83a7-f0340c44f34f-13-1-c000.csv,9036785
dbfs:/tmp/multiple.csv/part-00004-tid-7308919149863971029-29eda532-ce72-4f1b-83a7-f0340c44f34f-12-1-c000.csv,part-00004-tid-7308919149863971029-29eda532-ce72-4f1b-83a7-f0340c44f34f-12-1-c000.csv,9041847


### 파티셔닝
- <strong>어떤 데이터를 어디에 저장</strong>할 것인지 제어할 수 있는 기능
- <strong>디렉터리별로 컬럼 데이터를 인코딩</strong>해서 저장
  - 그래서 저장 후 읽을 때 전체 데이터셋을 스캔하지 않고 <strong>필요한 컬럼의 데이터만 읽기 가능</strong>
- 이 방식은 모든 파일 기반의 데이터소스에서 지원
- 파티셔닝은 <strong>필터링을 자주 사용하는 테이블</strong>을 가진 경우에 사용할 수 있는 가장 손쉬운 최적화 방식

In [0]:
#InvoiceNo값 기준으로 파티셔닝
df.limit(10).write.mode('overwrite').partitionBy('InvoiceNo').save('/tmp/partitioned-files.parquet')

In [0]:
display(df.limit(10))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850,United Kingdom
536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850,United Kingdom
536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850,United Kingdom
536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850,United Kingdom
536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047,United Kingdom


In [0]:
%fs ls /tmp/partitioned-files.parquet

path,name,size
dbfs:/tmp/partitioned-files.parquet/InvoiceNo=536365/,InvoiceNo=536365/,0
dbfs:/tmp/partitioned-files.parquet/InvoiceNo=536366/,InvoiceNo=536366/,0
dbfs:/tmp/partitioned-files.parquet/InvoiceNo=536367/,InvoiceNo=536367/,0
dbfs:/tmp/partitioned-files.parquet/_delta_log/,_delta_log/,0


---
- 각 폴더는 조건절을 폴더명으로 사용하며, 조건절을 만족하는 데이터가 저장된 파케이 파일을 가지고 있음

### 버켓팅
- <strong>각 파일에 저장된 데이터를 제어</strong>할 수 있는 <strong>파일 조직화</strong> 기법
- <strong>동일한 버킷 ID</strong>를 가진 데이터가 하나의 물리적 파티션에 모여 있으므로 데이터를 읽을 때 <strong>셔플을 피할 수 있음</strong>
  - 사전에 파티셔닝된 것이므로 조인이나 집계 시 발생하는 고비용의 셔플을 피할 수 있음
- <strong>높은 카디널리티(유니크 값이 많은)를 가지는 컬럼</strong>은 파티셔닝을 하면 수억 개의 디렉터리가 생성될 수도 있다.
  - 이때 버켓 단위로 데이터를 모아 일정 수의 파일로 저장하는 것이 효율적

<img src="https://miro.medium.com/max/1400/1*q4xHBk9ksw20Vf_25OYCtA.jpeg" width=70% />

- [참고 링크](https://blog.clairvoyantsoft.com/bucketing-in-spark-878d2e02140f)

In [0]:
df.select(F.countDistinct('InvoiceNo')).collect()

Out[16]: [Row(count(DISTINCT InvoiceNo)=25900)]

---
- 위 컬럼 기준으로 파티셔닝하면 25900개의 디렉터리가 생성될 것임
  - 아까는 limit(10)해서 3개밖에 생성되지 않은 것이고

In [0]:
#버켓단위로 데이터를 모아 일정 수의 파일로 저장하기
bucketNum = 10
bucketCol= 'InvoiceNo'

#기본적으로 아래 디렉터리 하위에 버켓팅 파일을 기록하므로 먼저 해당 디렉터리 생성
dbutils.fs.mkdirs('/user/hive/warehouse')

Out[9]: True

In [0]:
df.write.format('parquet').bucketBy(bucketNum, bucketCol).saveAsTable('bucketedFiles')

In [0]:
%fs ls /user/hive/warehouse/bucketedfiles/

path,name,size
dbfs:/user/hive/warehouse/bucketedfiles/_SUCCESS,_SUCCESS,0
dbfs:/user/hive/warehouse/bucketedfiles/_committed_1429916160167243299,_committed_1429916160167243299,8432
dbfs:/user/hive/warehouse/bucketedfiles/_started_1429916160167243299,_started_1429916160167243299,0
dbfs:/user/hive/warehouse/bucketedfiles/part-00000-tid-1429916160167243299-ae8dd407-c390-4806-8bb9-1eed6d0e2e7f-27-10_00009.c000.snappy.parquet,part-00000-tid-1429916160167243299-ae8dd407-c390-4806-8bb9-1eed6d0e2e7f-27-10_00009.c000.snappy.parquet,90262
dbfs:/user/hive/warehouse/bucketedfiles/part-00000-tid-1429916160167243299-ae8dd407-c390-4806-8bb9-1eed6d0e2e7f-27-1_00000.c000.snappy.parquet,part-00000-tid-1429916160167243299-ae8dd407-c390-4806-8bb9-1eed6d0e2e7f-27-1_00000.c000.snappy.parquet,87741
dbfs:/user/hive/warehouse/bucketedfiles/part-00000-tid-1429916160167243299-ae8dd407-c390-4806-8bb9-1eed6d0e2e7f-27-2_00001.c000.snappy.parquet,part-00000-tid-1429916160167243299-ae8dd407-c390-4806-8bb9-1eed6d0e2e7f-27-2_00001.c000.snappy.parquet,74086
dbfs:/user/hive/warehouse/bucketedfiles/part-00000-tid-1429916160167243299-ae8dd407-c390-4806-8bb9-1eed6d0e2e7f-27-3_00002.c000.snappy.parquet,part-00000-tid-1429916160167243299-ae8dd407-c390-4806-8bb9-1eed6d0e2e7f-27-3_00002.c000.snappy.parquet,84847
dbfs:/user/hive/warehouse/bucketedfiles/part-00000-tid-1429916160167243299-ae8dd407-c390-4806-8bb9-1eed6d0e2e7f-27-4_00003.c000.snappy.parquet,part-00000-tid-1429916160167243299-ae8dd407-c390-4806-8bb9-1eed6d0e2e7f-27-4_00003.c000.snappy.parquet,92241
dbfs:/user/hive/warehouse/bucketedfiles/part-00000-tid-1429916160167243299-ae8dd407-c390-4806-8bb9-1eed6d0e2e7f-27-5_00004.c000.snappy.parquet,part-00000-tid-1429916160167243299-ae8dd407-c390-4806-8bb9-1eed6d0e2e7f-27-5_00004.c000.snappy.parquet,89909
dbfs:/user/hive/warehouse/bucketedfiles/part-00000-tid-1429916160167243299-ae8dd407-c390-4806-8bb9-1eed6d0e2e7f-27-6_00005.c000.snappy.parquet,part-00000-tid-1429916160167243299-ae8dd407-c390-4806-8bb9-1eed6d0e2e7f-27-6_00005.c000.snappy.parquet,84077


### 복합 데이터 유형 쓰기
- csv파일은 복합 데이터 타입을 지원하지 않음
- 파케이, ORC는 복합 데이터 타입 지원

### 파일 크기 관리
- 데이터를 저장할 땐 파일 크기가 중요한 요소가 아니지만 읽을 땐 중요한 요소 중 하나

- 작은 크기의 파일 문제
  - <strong>작은 파일을 많이 생성</strong>하면 메타데이터에 엄청난 <strong>관리 부하</strong>가 발생
  - HDFS 등 많은 파일 시스템은 작은 크기의 파일을 잘 다루지 못함
  - 스파크는 특히 더 못다룸
  
- 큰 크기의 파일 문제
  - 몇 개의 로우가 필요하더라도 전체 데이터 블록을 읽어야하므로 비효율적
  
- 스파크 2.2버전에는 파일 크기를 제어할 수 있는 새로운 방법이 도입됨
  - <strong>maxRecordsPerFile</strong>옵션에 파일당 레코드 수를 지정하여 파일 크기를 효과적으로 제어 가능

In [0]:
#파일당 최대 5000개의 로우를 포함하도록 보장
df.write.option('maxRecordsPerFile',5000)

Out[17]: <pyspark.sql.readwriter.DataFrameWriter at 0x7fcf15185eb0>